In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
%matplotlib inline

import os
import sys
import datetime

# import pyvips as pv
# import wandb
import cv2

import warnings; warnings.filterwarnings('ignore')

sys.path.append('..')
from config import CFG
CFG = CFG()
pd.options.display.float_format = '{:,.2f}'.format
pd.set_option('display.max_columns', CFG.NCOLS)
pd.set_option('display.max_rows', CFG.NROWS)

In [2]:
def resize_image(image, current_mpp, target_mpp):
    # Calculate the resizing factor
    resizing_factor = current_mpp / target_mpp
    width = int(image.shape[1] * resizing_factor)
    height = int(image.shape[0] * resizing_factor)
    dim = (width, height)

    return cv2.resize(image, dim, interpolation = cv2.INTER_AREA)

In [3]:
train_meta = pd.read_csv(f'{CFG.DATA_PATH}/train_metadata.csv')
test_meta = pd.read_csv(f'{CFG.DATA_PATH}/test_metadata.csv')
train_labels = pd.read_csv(f'{CFG.DATA_PATH}/train_labels.csv')
test_labels = pd.read_csv(f'{CFG.DATA_PATH}/test_labels.csv')
annot = pd.read_csv(f'{CFG.DATA_PATH}/annotations.csv')

In [4]:
hires = train_meta[train_meta['resolution'] == 0.121399]
hires.to_csv('../data/hires.csv', index=False)
train_meta[~train_meta['filename'].isin(hires['filename'].values.tolist())].to_csv('../data/lowres.csv', index=False)

In [26]:
tmp = pd.read_csv('../workspace/tile_data/48/0e1v575k-tile_data.csv', header=12).sort_values('Score', ascending=False).reset_index(drop=True)
tmp[tmp['Score'] > 0.7]

,Tile Num,Row,Column,Tissue %,Tissue Quantity,Col Start,Row Start,Col End,Row End,Col Size,Row Size,Color Factor,S and V Factor,Quantity Factor,Score
0,4560,50,52,94.10,HIGH,2448,2352,2496,2400,48,48,741,1.00,1.00,0.85
1,779,9,43,86.76,HIGH,2016,384,2064,432,48,48,1871,1.00,1.00,0.85
2,4561,50,53,91.67,HIGH,2496,2352,2544,2400,48,48,668,1.00,1.00,0.85
3,868,10,40,90.15,HIGH,1872,432,1920,480,48,48,791,1.00,1.00,0.84
4,871,10,43,97.70,HIGH,2016,432,2064,480,48,48,590,0.49,1.00,0.84
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
127,867,10,39,68.92,MEDIUM,1824,432,1872,480,48,48,758,1.00,0.20,0.70
128,3081,34,45,61.98,MEDIUM,2112,1584,2160,1632,48,48,2488,1.00,0.20,0.70
129,2865,32,13,81.81,HIGH,576,1488,624,1536,48,48,33,1.00,1.00,0.70
130,4382,48,58,67.27,MEDIUM,2736,2256,2784,2304,48,48,874,1.00,0.20,0.70


In [71]:
!s5cmd --no-sign-request ls s3://drivendata-competition-visiomel-public-us/images/ > list.txt
with open('list.txt') as f:
    l = f.readlines()
    l = l[1:]
l = pd.Series(l).apply(lambda x: x.split(' ')[-1].split('.')[0])
dl = os.listdir(f'{CFG.DATA_PATH}/images')
dl = pd.Series(dl).apply(lambda x: x.split('.')[0])
dlist = l[~l.isin(dl)]
dlist = dlist.apply(lambda x: x + '.tif')
train_meta = train_meta[train_meta['filename'].isin(dlist.values.tolist())]

In [3]:
def dl_img(row):
    path = row[10]
    file = row[1]
    new = row[1].split('.')[0]
    !s5cmd --no-sign-request cp {path} /mnt/disk1/visiomel-melanoma-runtime/data/images/

    img = pv.Image.new_from_file(f'{CFG.DATA_PATH}/images/{file}', page=4)
    img.write_to_file(f'{CFG.DATA_PATH}/images/{new}.jpg')
    del img
    os.remove(f'{CFG.DATA_PATH}/images/{file}')
    gc.collect()

In [3]:
def dl_img(dlpath):
    fname = dlpath.split('/')[-1]
    !s5cmd --no-sign-request cp {dlpath} /mnt/disk1/visiomel-melanoma-runtime/data/originals/
    !s5cmd --endpoint-url https://storage.googleapis.com cp /mnt/disk1/visiomel-melanoma-runtime/data/originals/{fname} s3://visiomel/originals/

In [4]:
import gc
for row in train_meta.itertuples():
    dl_img(row[10])

cp s3://drivendata-competition-visiomel-public-us/images/1u4lhlqb.tif /mnt/disk1/visiomel-melanoma-runtime/data/originals/1u4lhlqb.tif
cp /mnt/disk1/visiomel-melanoma-runtime/data/originals/1u4lhlqb.tif s3://visiomel/originals/1u4lhlqb.tif
cp s3://drivendata-competition-visiomel-public-us/images/rqumqnfp.tif /mnt/disk1/visiomel-melanoma-runtime/data/originals/rqumqnfp.tif
cp /mnt/disk1/visiomel-melanoma-runtime/data/originals/rqumqnfp.tif s3://visiomel/originals/rqumqnfp.tif
cp s3://drivendata-competition-visiomel-public-us/images/bu5xt1xm.tif /mnt/disk1/visiomel-melanoma-runtime/data/originals/bu5xt1xm.tif
cp /mnt/disk1/visiomel-melanoma-runtime/data/originals/bu5xt1xm.tif s3://visiomel/originals/bu5xt1xm.tif
cp s3://drivendata-competition-visiomel-public-us/images/dibvu7wk.tif /mnt/disk1/visiomel-melanoma-runtime/data/originals/dibvu7wk.tif
cp /mnt/disk1/visiomel-melanoma-runtime/data/originals/dibvu7wk.tif s3://visiomel/originals/dibvu7wk.tif
cp s3://drivendata-competition-visiomel-